# Zadanie: Implementacja RAG z bazą SQL biura podróży

## Opis zadania

Stwórz system RAG (Retrieval-Augmented Generation) wykorzystujący bazę danych SQL biura podróży. System ma analizować zapytanie użytkownika, określać jakie informacje musi pobrać z bazy danych, a następnie wykorzystywać te informacje do wygenerowania odpowiedzi.

Wykorzystaj bazę danych biura podróży w SQLite (`008-03. RAG data/biuro_podrozy.db`)

Działaj z perspektywy __2025-05-01__ (przekaż tę datę modeli w system prompcie jako aktualną).

## Potencjalne podejście 1

1. Wykorzystanie LLM do przeanalizowania zapytania użytkownika, i stworzenia zapytania SQL, które wyciągnie z bazy potrzebne informacje.
2. Wykonanie zapytań na bazie danych.
3. Wykorzystanie LLM do sformułowania finalnej odpowiedzi, w oparciu o pytanie użytkownika i wynik zapytań.

## Potencjalne podejście 2

1. Wykorzystanie LLM do obsługi zapytania użytkownika.
2. Udostępnienie mu narzędzia do wysyłania SELECTów do bazy danych.
    + więcej w temacie Function Callingu robiliśmy w [tym notebooku](004-01.%20LLM-structured_outputs.ipynb)

## Schemat bazy danych

```{sql}
CREATE TABLE clients (
      client_id INTEGER PRIMARY KEY,
      first_name TEXT NOT NULL,
      last_name TEXT NOT NULL,
      email TEXT NOT NULL UNIQUE,
      phone TEXT,
      address TEXT,
      city TEXT,
      postal_code TEXT,
      date_of_birth DATE,
      registration_date DATE NOT NULL,
      preferred_payment_method TEXT,
      loyalty_points INTEGER DEFAULT 0
   )

   CREATE TABLE destinations (
      destination_id INTEGER PRIMARY KEY,
      name TEXT NOT NULL,
      country TEXT NOT NULL,
      description TEXT,
      climate TEXT,
      best_time_to_visit TEXT,
      language TEXT,
      currency TEXT,
      popular_score INTEGER,
      average_temperature REAL
   )

   CREATE TABLE hotels (
      hotel_id INTEGER PRIMARY KEY,
      destination_id INTEGER NOT NULL,
      name TEXT NOT NULL,
      stars INTEGER CHECK (stars BETWEEN 1 AND 5),
      description TEXT,
      amenities TEXT,
      room_types TEXT,
      distance_to_center REAL,
      price_per_night REAL,
      FOREIGN KEY (destination_id) REFERENCES destinations (destination_id)
   )

   CREATE TABLE tours (
      tour_id INTEGER PRIMARY KEY,
      destination_id INTEGER NOT NULL,
      name TEXT NOT NULL,
      description TEXT,
      duration_days INTEGER,
      included_services TEXT,
      activities TEXT,
      difficulty_level TEXT,
      base_price REAL,
      availability_start DATE,
      availability_end DATE,
      FOREIGN KEY (destination_id) REFERENCES destinations (destination_id)
   )

   CREATE TABLE employees (
      employee_id INTEGER PRIMARY KEY,
      first_name TEXT NOT NULL,
      last_name TEXT NOT NULL,
      position TEXT NOT NULL,
      department TEXT,
      email TEXT NOT NULL UNIQUE,
      phone TEXT,
      hire_date DATE,
      salary REAL
   )

   CREATE TABLE bookings (
      booking_id INTEGER PRIMARY KEY,
      client_id INTEGER NOT NULL,
      tour_id INTEGER,
      hotel_id INTEGER,
      employee_id INTEGER,
      booking_date DATE NOT NULL,
      travel_start_date DATE NOT NULL,
      travel_end_date DATE NOT NULL,
      num_adults INTEGER NOT NULL,
      num_children INTEGER DEFAULT 0,
      total_price REAL NOT NULL,
      status TEXT NOT NULL,
      special_requirements TEXT,
      FOREIGN KEY (client_id) REFERENCES clients (client_id),
      FOREIGN KEY (tour_id) REFERENCES tours (tour_id),
      FOREIGN KEY (hotel_id) REFERENCES hotels (hotel_id),
      FOREIGN KEY (employee_id) REFERENCES employees (employee_id)
   )

   CREATE TABLE payments (
      payment_id INTEGER PRIMARY KEY,
      booking_id INTEGER NOT NULL,
      payment_date DATE NOT NULL,
      amount REAL NOT NULL,
      payment_method TEXT NOT NULL,
      status TEXT NOT NULL,
      transaction_id TEXT,
      FOREIGN KEY (booking_id) REFERENCES bookings (booking_id)
   )

   CREATE TABLE reviews (
      review_id INTEGER PRIMARY KEY,
      booking_id INTEGER NOT NULL,
      client_id INTEGER NOT NULL,
      title TEXT,
      content TEXT,
      rating INTEGER CHECK (rating BETWEEN 1 AND 10),
      review_date DATE NOT NULL,
      positive_aspects TEXT,
      negative_aspects TEXT,
      FOREIGN KEY (booking_id) REFERENCES bookings (booking_id),
      FOREIGN KEY (client_id) REFERENCES clients (client_id)
   )
```

## Importowanie niezbędnych bibliotek

In [ ]:
import sqlite3
import pandas as pd

## Połączenie z bazą danych SQLite

Łączymy się z bazą danych biura podróży i wyświetlamy dostępne tabele.

In [ ]:
# Ścieżka do bazy danych
db_path = "008-03. RAG data/biuro_podrozy.db"
# Łączenie z bazą danych w trybie tylko do odczytu
conn = sqlite3.connect(f"file:{db_path}?mode=ro", uri=True)

In [ ]:
# Przykładwego zapytania do bazy danych
random_hotels_query = """SELECT h.name AS hotel_name, h.stars, d.name AS destination, d.country, h.price_per_night 
                      FROM hotels h 
                      JOIN destinations d ON h.destination_id = d.destination_id 
                      ORDER BY RANDOM() 
                      LIMIT 3;"""

random_hotels = pd.read_sql_query(random_hotels_query, conn)

random_hotels

--- WASZ KOD ---

--- WASZ KOD ---

## Przetestujmy nasz system RAG

Zadajmy kilka przykładowych pytań, aby sprawdzić działanie systemu.

In [ ]:
# Przykładowe pytanie 1
question = "Jakie są najpopularniejsze destynacje w naszej ofercie?"
answer = answer_travel_query(question)
print("\nOdpowiedź:")
print(answer)

In [ ]:
# Przykładowe pytanie 2
question = "Które hotele mają najwyższe oceny i gdzie się znajdują?"
answer = answer_travel_query(question)
print("\nOdpowiedź:")
print(answer)

In [ ]:
# Przykładowe pytanie 3
question = "Jakie wycieczki są dostępne w lecie i ile kosztują?"
answer = answer_travel_query(question)
print("\nOdpowiedź:")
print(answer)

In [ ]:
# Zamknięcie połączenia z bazą danych
conn.close()